## Project 6

In [40]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv
import psycopg2
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from types import NoneType

%matplotlib inline

NUMREVIEWS = 3 # If you want more movie reviews loaded for model testing/adjustment, change this value here, and re-run the code
DATALOAD = 'CSV'  # If you want to load using IMDB API, change to 'IMDB'.  NOTE:  IMDB load takes a long time.

### Pre-Work: Write a problem statement 

## Part 1: Acquire the Data

#### 1. Connect to the IMDB API

In [42]:
if (DATALOAD == 'IMDB'):
    imdb = Imdb()
    imdb = Imdb(anonymize=True)

#### 2. Query the top 250 rated movies in the database

In [43]:
if (DATALOAD == 'IMDB'):
    MoviesList = imdb.top_250()
    if len(MoviesList) == 250 :
        print 'Successful load of the Top 250 Movies.'
    else:
        print 'Failed to load Top 250 Movies.'
else: # Load movie and review data from CSV file
    dfMovies = pd.read_csv(filepath_or_buffer="Top25Movies.csv", sep='\t',na_values='')
    dfReviews = pd.read_csv(filepath_or_buffer="MovieReviews.csv", sep='\t',na_values='')
    print 'Successful load of the Movie & Review data from CSV file.'

Successful load of the Movie & Review data from CSV file.


#### 3. Only select the top 25 movies and delete the uncessary rows

In [44]:
## Credit to Paul Trichon for the research on this section of code.

if (DATALOAD == 'IMDB'):
    # Initialize variables
    counter=0
    moviesDictList = {"title_id":[],
                  "title": [],
                  "year":[],
                  "type":[],
                  "rating":[],
                  "total_votes":[]}
    reviewsDictList = {"title_id":[],
                      "username":[],
                      "review_text":[],
                      "review_date":[],
                      "summary":[],
                      "status":[],
                      "user_location":[],
                      "user_score":[],
                      "user_score_count":[]}

    # Cycle through the top 25 movies.
    for movie in MoviesList:
        counter += 1
        # When we have loaded 25 movies, break out of the loop.
        if counter > 25:
            break

        # Load the movie information into a movieDict variable.
        title_id = movie["tconst"]
        moviesDictList["title_id"].append(title_id)
        moviesDictList["title"].append(movie["title"].encode('utf-8').strip())
        moviesDictList["year"].append(movie["year"])
        moviesDictList["type"].append(movie["type"].encode('utf-8').strip())
        moviesDictList["rating"].append(movie["rating"])
        moviesDictList["total_votes"].append(movie["num_votes"])

        # For each movie, get NUMREVIEWS reviews.
        # NOTE:  If you want to change the number of reviews retrieved, please do so in Cell #1.
        reviewsList = imdb.get_title_reviews(title_id,max_results = NUMREVIEWS) 
        # Load each review into a reviews data dictionary.
        for review in reviewsList:
            reviewsDictList["title_id"].append(title_id)
            reviewsDictList["username"].append(review.username)
            reviewsDictList["review_text"].append(review.text)
            reviewsDictList["review_date"].append(review.date)
            reviewsDictList["summary"].append(review.summary)
            reviewsDictList["status"].append(review.status)
            reviewsDictList["user_location"].append(review.user_location)
            reviewsDictList["user_score"].append(review.user_score)
            reviewsDictList["user_score_count"].append(review.user_score_count)

In [45]:
if (DATALOAD == 'IMDB'):
    # Load the Movies into a dataframe.
    dfMovies = pd.DataFrame(data=moviesDictList,columns=["title_id","title","year","type","rating","total_votes"])

    # Load the Reviews into a dataframe.
    dfReviews = pd.DataFrame(data=reviewsDictList,\
                             columns=["title_id","status","review_date","review_text","summary","username",\
                                      "user_location", "user_score","user_score_count"])

In [46]:
dfMovies.head()

,title_id,title,year,type,rating,total_votes
0,tt0111161,The Shawshank Redemption,1994,feature,9.3,1678314
1,tt0068646,The Godfather,1972,feature,9.2,1148275
2,tt0071562,The Godfather: Part II,1974,feature,9.0,785197
3,tt0468569,The Dark Knight,2008,feature,9.0,1663494
4,tt0108052,Schindlers List,1993,feature,8.9,858896


In [47]:
dfReviews.head()

,title_id,status,review_date,review_text,summary,username,user_location,user_score,user_score_count
0,tt0111161,G,2003-11-26,Why do I want to write the 234th comment on Th...,Tied for the best movie I have ever seen,carflo,Texas,2102,2421
1,tt0111161,G,2002-08-27,"Can Hollywood, usually creating things for ent...",Shawshank Redeems Hollywood,Wesley S. Walker,"Paducah, KY",1380,1613
2,tt0111161,G,2001-02-08,I have never seen such an amazing film since I...,Prepare to be moved,speedreid,"Westerville, Ohio",1248,1449
3,tt0111161,S,2006-02-10,"In its Oscar year, Shawshank Redemption (writt...",A classic piece of unforgettable film-making.,Justin M (kaspen12),"Vancouver, Canada",876,992
4,tt0111161,S,1998-08-16,The reason I became a member of this database ...,Two movies in one,Douglas-2,"Chicago, IL",920,1141


In [48]:
if (DATALOAD == 'IMDB'):
    # We are going to do some data scrubbing!!

    # Start with the Movies.
    # We are going to conver the string values to 'utf-8' format, remove whitespace characters.
    dfMovies["title_id"] = [x.encode('utf-8') for x in dfMovies["title_id"]]
    dfMovies["title"] = [x.encode('utf-8').strip().replace("\r","").replace("'","").replace('\n',"")\
                         .replace('"',"") for x in dfMovies["title"]]
    dfMovies["year"] = [int(x) for x in dfMovies["year"]]
    dfMovies["type"] = [x.encode('utf-8') for x in dfMovies["type"]]
    dfMovies["rating"] = [float(x) for x in dfMovies["rating"]]
    dfMovies["total_votes"] = [int(x) for x in dfMovies["total_votes"]]
    dfMovies

In [49]:
if (DATALOAD == 'IMDB'):
    # Scrub a dub dub!  Now on to clean the Reviews.
    dfReviews["title_id"] = [x.encode('utf-8') for x in dfReviews["title_id"]]
    dfReviews["status"] = [x.encode('utf-8') for x in dfReviews["status"]]
    dfReviews["review_date"] = pd.to_datetime(arg=dfReviews["review_date"],format='%Y-%m-%d')
    dfReviews["review_text"] = [x.encode('utf-8').strip().replace("\r","").replace('\n',"") for x in dfReviews["review_text"]]
    dfReviews["summary"] = ['' if (isinstance(x, NoneType) == True) else x.encode('utf-8').strip().replace("\r","").replace('\n',"") for x in dfReviews["summary"]]
    dfReviews["username"] = [x.encode('utf-8') for x in dfReviews["username"]]
    dfReviews["user_location"] = ['' if (isinstance(x, NoneType) == True) else x.encode('utf-8').strip().replace("\r","").replace('\n',"") for x in dfReviews["user_location"]]
    dfReviews["user_score"] = [int(x) for x in dfReviews["user_score"]]
    dfReviews["user_score_count"] = [int(x) for x in dfReviews["user_score_count"]]
    dfReviews

#### 4. Write the Results to a csv

In [50]:
if (DATALOAD == 'IMDB'):
    # Because of the nature of our data - specifically the text in the reviews - it is best to save the data to a tab-separated
    # file.  This will be the best way to maintain the integrity of the data as much as possible.
    dfMovies.to_csv('Top25Movies.csv',sep='\t', index=False, encoding='utf-8', \
                    columns=["title_id","title","year","type","rating","total_votes"])
    dfReviews.to_csv('MovieReviews.csv',sep='\t', index=False, encoding='utf-8', quotechar='"', \
                    columns=["title_id","status","review_date","review_text","summary","username",\
                             "user_location", "user_score","user_score_count"])

## Part 2: Wrangle the text data

#### 1. Convert the listing identification numbers (tconst) from the first dataframe to a list

In [51]:
# See below...

#### 2. Scrape the reviews for the top 25 movies

*Hint*: Use a loop to scrape each page at once

In [52]:
# Silly!  I already did that!

#### 3. Work through each title and find the most common descriptors

*Hint*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements

In [53]:
# I am not sure what this is asking for...Is it referring to most frequent words in our Top 25 movie titles, or is this
# referring to web scraping activities?

#### 4. Convert to a string and remove the non AlphaNumeric characters

*Hint*: Use regular expressions!

In [54]:
#  Thanks for the hint!  It worked splendidly.
regex = re.compile('[^a-z A-Z0-9]')
dfReviews["review_text"] = [regex.sub('', str(x)) for x in dfReviews["review_text"]]
dfReviews["summary"] = [regex.sub('', str(x)) for x in dfReviews["summary"]]
dfReviews.head()

,title_id,status,review_date,review_text,summary,username,user_location,user_score,user_score_count
0,tt0111161,G,2003-11-26,Why do I want to write the 234th comment on Th...,Tied for the best movie I have ever seen,carflo,Texas,2102,2421
1,tt0111161,G,2002-08-27,Can Hollywood usually creating things for ente...,Shawshank Redeems Hollywood,Wesley S. Walker,"Paducah, KY",1380,1613
2,tt0111161,G,2001-02-08,I have never seen such an amazing film since I...,Prepare to be moved,speedreid,"Westerville, Ohio",1248,1449
3,tt0111161,S,2006-02-10,In its Oscar year Shawshank Redemption written...,A classic piece of unforgettable filmmaking,Justin M (kaspen12),"Vancouver, Canada",876,992
4,tt0111161,S,1998-08-16,The reason I became a member of this database ...,Two movies in one,Douglas-2,"Chicago, IL",920,1141


#### 5. Tokenize the Output

In [55]:
cvec = CountVectorizer(stop_words='english')
cvec.fit(dfReviews['review_text'])
print "Feature Count: " + str(len(cvec.get_feature_names()))

Feature Count: 15905


#### 6. Convert to a Dataframe for Easy Viewing

In [56]:
X_train = pd.DataFrame(cvec.transform(dfReviews['review_text']).todense(),
                       columns=cvec.get_feature_names())
X_train.head()

,007like,10,100,1000,10000,100s,101,1010,1010thanks,1011,...,zimmer,zimmers,zipping,zodiac,zone,zones,zoom,zooming,zooms,zulu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


####  7. Find the rows with the top five descriptive words

In [57]:
words = ('best', 'hope', 'love', 'beautiful', 'great')

In [58]:
# Here are the 10 Most Frequently used words in the reviews we scraped
SWordFreq = pd.Series(data=([sum([x for x in X_train[c]]) for c in X_train.columns]),index=X_train.columns)
SWordFreq.sort_values(ascending=False,inplace=True)
SWordFreq[:10]

film     1789
movie    1182
just      544
time      496
like      493
best      444
story     437
films     415
great     387
good      364
dtype: int64

In [59]:
# Find the frequencies of the list of words.
for i in words:
    print "Frequency Count for '" + i + "': " + str(SWordFreq[i])

Frequency Count for 'best': 444
Frequency Count for 'hope': 94
Frequency Count for 'love': 117
Frequency Count for 'beautiful': 56
Frequency Count for 'great': 387


#### 8. Write the results to a csv

In [60]:
X_train.to_csv("TokenMovieReviews.csv")

#### 9. Repeat the process for the other top 24 titles

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

In [61]:
#  Connecting to a database using psycopg2
conn = psycopg2.connect('host=localhost dbname=dsiMovies user=dsi_student password=gastudents port=5432')

#### 3. Join the two tables 

In [62]:
sql = 'SELECT m.*, mr.* FROM movies as m JOIN movie_reviews as mr ON m.title_id = mr.title_id;'

#### 4. Select the newly joined table and save two copies of the into dataframes

In [67]:
dfResults1 = pd.read_sql_query(sql,con=conn)
dfResults2 = pd.read_sql_query(sql,con=conn)
dfResults1.shape

(500, 15)

In [68]:
dfResults1.head()

,title_id,title,movie_year,movie_type,rating,total_votes,title_id,status,review_date,review_text,summary,review_username,user_location,user_score,user_score_count
0,tt0111161,The Shawshank Redemption,1994,feature,9.3,1678314,tt0111161,G,2003-11-26,Why do I want to write the 234th comment on Th...,Tied for the best movie I have ever seen,carflo,Texas,2102,2421
1,tt0111161,The Shawshank Redemption,1994,feature,9.3,1678314,tt0111161,G,2002-08-27,"Can Hollywood, usually creating things for ent...",Shawshank Redeems Hollywood,Wesley S. Walker,"Paducah, KY",1380,1613
2,tt0111161,The Shawshank Redemption,1994,feature,9.3,1678314,tt0111161,G,2001-02-08,I have never seen such an amazing film since I...,Prepare to be moved,speedreid,"Westerville, Ohio",1248,1449
3,tt0111161,The Shawshank Redemption,1994,feature,9.3,1678314,tt0111161,S,2006-02-10,"In its Oscar year, Shawshank Redemption (writt...",A classic piece of unforgettable film-making.,Justin M (kaspen12),"Vancouver, Canada",876,992
4,tt0111161,The Shawshank Redemption,1994,feature,9.3,1678314,tt0111161,S,1998-08-16,The reason I became a member of this database ...,Two movies in one,Douglas-2,"Chicago, IL",920,1141


## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename the column headings

#### 2. Run a description of the data

#### 3. Visualize the Data

## Part 3: Build the Decision Tree

#### 1. What is our target attribute? 

#### 2. Prepare the data and define the training set

#### 2. Train the Model

#### 3. Set up test data and test the model

#### 5. Check the results

#### 6. What is overfitting and how are we at risk? 